In [3]:
from typing import TypedDict

from langchain.chat_models import init_chat_model
from langchain_community.chat_models.tongyi import ChatTongyi
from langgraph.graph import START, StateGraph

# The joke_model is tagged with "joke"
# joke_model = init_chat_model(model="qwen-max", tags=["joke"])
# The poem_model is tagged with "poem"
# poem_model = init_chat_model(model="qwen-max", tags=["poem"])
joke_model = ChatTongyi(model="qwen-max", tags=["joke"], streaming=True)
poem_model = ChatTongyi(model="qwen-max", tags=["poem"], streaming=True)


class State(TypedDict):
    topic: str
    joke: str
    poem: str


async def call_model(state, config):
    topic = state["topic"]
    print("Writing joke...")
    joke_response = await joke_model.ainvoke(
        [{"role": "user", "content": f"Write a joke about {topic}"}],
        config,
    )
    print("\n\nWriting poem...")
    poem_response = await poem_model.ainvoke(
        [{"role": "user", "content": f"Write a short poem about {topic}"}],
        config,
    )
    return {"joke": joke_response.content, "poem": poem_response.content}


graph = (
    StateGraph(State)
    .add_node(call_model)
    .add_edge(START, "call_model")
    .compile()
)

In [4]:

# The stream_mode is set to "messages" to stream LLM tokens
# The metadata contains information about the LLM invocation, including the tags
async for msg, metadata in graph.astream(
    {"topic": "cats"},
    stream_mode="messages",
):
    if metadata["tags"] == ["joke"]:
        print(msg.content, end="|", flush=True)

Writing joke...
Why| did the cat| join| the| gym? Because it wanted to| improve its mew|-scles!|||

Writing poem...
